In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [8]:
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data
** Create a new length feature: **

In [11]:
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length',length(data['text']))

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [15]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [17]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [20]:
from pyspark.ml import Pipeline

In [21]:
data_prep_pipe = Pipeline(stages =[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [23]:
cleaner = data_prep_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [25]:
clean_data = clean_data.select(['label', 'features'])

In [26]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [27]:
(training, testing) = clean_data.randomSplit([0.7,0.3])

In [28]:
spam_predictor = nb.fit(training)

In [29]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [30]:
test_result = spam_predictor.transform(testing)

In [31]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,5,20,...|[-809.88044407652...|[1.0,1.0455943341...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.55419705377...|[1.0,3.7086805575...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1360.6565383597...|[1.0,1.3169873512...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-838.20074580167...|[1.0,1.6462673032...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-968.97200699920...|[1.0,3.5070642981...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1009.3217485653...|[1.0,7.2742503214...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1489.3326993613...|[0.99999999839238...|       0.0|
|  0.0|(13424,[0,1,30,12...|[-617.37867679533...|[1.0,1.0651882867...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-615.81752734833...|[0.96056796780580...|       0.0|
|  0.0|(13424,[0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_result)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9183258173895135
